In [21]:
from constants import *
import subprocess
import json
import os
from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI
from eth_account import Account

load_dotenv()

True

In [22]:
mnemonic = os.getenv('MNEMONIC')
print(mnemonic)

reform metal behind party exhaust place length during you cram arena add


In [24]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer =0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [39]:
def derive_wallets (mnemonic, coin, num):
    command = f'./derive -g --mnemonic="{mnemonic}" --coin="{coin}" --numderive="{num}" --cols=index,path,address,privkey,pubkey,pubkeyhash,xprv,xpub --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [40]:
derive_wallets(mnemonic, ETH, 5)

[{'index': 0,
  'path': "m/44'/60'/0'/0/0",
  'address': '0xF7F66a3883d2Dce1887f623E4Bfa68BADd09e13C',
  'privkey': '0xf30006b745f00e77350713d2335f3c5fa91d693b6e8de219ffee26bfbd94fed3',
  'pubkey': '03466dd9f4a76b733fea9058b369606013005e8d4f114ce40e9fff56300c572678',
  'pubkeyhash': 'b0e61450f60134468e621bdc61e4ab613e213678',
  'xprv': 'xprvA3y3sHa3MwtdAXpxC6unBw8a4jVMLPwJJ9vW6BEpgzT4GiMGaJEQAeDo3oJgKSDk8TKs5U4mkXH1Ys9c2Bv7HXoPRLMmiNCu6iuvWsxyyFn',
  'xpub': 'xpub6GxQGo6wCKSvP1uRJ8SnZ55JcmKqjrf9fNr6tZeSFKz39WgR7qYeiSYGu4yBqpwBy3sERnRFYfPKYsE5rtAaSGny4ivgZQ4k9pMvQSrWSLp'},
 {'index': 1,
  'path': "m/44'/60'/0'/0/1",
  'address': '0xaB30a1260dD2db95653d7e8c82712057330a8B17',
  'privkey': '0x0adc3fc19a7c6e3f06bc01b876c7655318f72fc6afedf75252c2547bdf4e69da',
  'pubkey': '03f7caf442b99332966e6ab6c76eb5ad280aaa0e766c4d539d174f43f4a0f6f3fa',
  'pubkeyhash': '33379f3ff99362dd52aa93e0fa6207e1f7a8a6ee',
  'xprv': 'xprvA3y3sHa3MwtdEByoSgo1uhf3MJNQNjVt5aofrj6CZJbkmErGru49suUNLpj32VcvWm8mtpn44bLMoY

In [44]:
def priv_key_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)


In [45]:
ETH_key = priv_key_account(ETH, '0xe68166b846ec0245ca9efd39a3deeff2008d27d38e4b9c4660fd7e4e5989cf64')

In [46]:
ETH_key

In [47]:
def create_trx(coin, account, recipient, amount):
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)]) 

In [48]:
trx = create_trx(ETH, ETH_key, '0xF0f66976D133d1AE93B2131bBBA0C8B913d895F6', 1)

In [49]:
trx

{'to': '0xF0f66976D133d1AE93B2131bBBA0C8B913d895F6',
 'from': '0xC6d5487ba0Bda90DAe66f20B0FEb988De9A9e1dF',
 'value': 1,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 0}

In [52]:
def send_tx(coin,account, recipient, amount):
    tx = create_trx(coin,account,recipient,amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [53]:
send_tx(ETH, ETH_key, '0xF0f66976D133d1AE93B2131bBBA0C8B913d895F6', 1)

ValueError: {'message': "sender doesn't have enough funds to send tx. The upfront cost is: 420000000000001 and the sender's account only has: 0", 'code': -32000, 'data': {'stack': "Error: sender doesn't have enough funds to send tx. The upfront cost is: 420000000000001 and the sender's account only has: 0\n    at VM.<anonymous> (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache-core/node_modules/ethereumjs-vm/lib/runTx.ts:114:11)\n    at step (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache-core/node_modules/ethereumjs-vm/dist/runTx.js:32:23)\n    at Object.next (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache-core/node_modules/ethereumjs-vm/dist/runTx.js:13:53)\n    at fulfilled (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache-core/node_modules/ethereumjs-vm/dist/runTx.js:4:58)\n    at runMicrotasks (<anonymous>)\n    at processTicksAndRejections (internal/process/task